In [186]:
from tools import *
import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, GATConv
from tqdm import tqdm  


Définition du modèle 

In [187]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, num_features,output_layer, hidden_dim=128,hidden_dim_2=128,hidden_dim_3=128):
        super().__init__()
        self.conv1 = SAGEConv(num_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim_2)
        self.conv3 = SAGEConv(hidden_dim_2, hidden_dim_3)
        self.conv4 = SAGEConv(hidden_dim_2, hidden_dim_3)
        self.conv5 = SAGEConv(hidden_dim_2, hidden_dim_3)

         
        self.lin = torch.nn.Linear(hidden_dim_3, output_layer)

    def forward(self, data,dropout=0.2):
        x, edge_index = data.x[:,:4], data.edge_index
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.dropout(x, p=dropout, train=self.training)
        x = torch.relu(self.conv2(x, edge_index))
        x = torch.dropout(x, p=dropout, train=self.training)
        x = torch.relu(self.conv3(x, edge_index))
        x = torch.dropout(x, p=dropout, train=self.training)
        # x = torch.relu(self.conv4(x, edge_index))
        # x = torch.dropout(x, p=dropout, train=self.training)
        # x = torch.relu(self.conv5(x, edge_index))
        # x = torch.dropout(x, p=dropout, train=self.training)

        x = self.lin(x)
        return x



Fonction d' Entrainement

In [189]:
def train(model,epochs,train_loader,optimizer):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_correct = 0
        total_samples = 0

        for data in train_loader:
            data = data.to(torch.device('cpu'))
            optimizer.zero_grad()
            out = model(data)
            targets = data.y.long()
            loss = F.cross_entropy(out, targets)
            loss.backward()
            optimizer.step()
            _, pred = out.max(dim=-1)
            correct = (pred == targets).sum().item()
            total_loss += loss.item()
            total_correct += correct
            total_samples += targets.size(0)

        if (epoch % 10 == 0) or (epoch == 99):
            epoch_loss = total_loss / len(train_loader)
            epoch_acc = total_correct / total_samples
            print(f'Epoch {epoch+1:3d} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')


Fonction de test

In [192]:
def evaluate(model, graphs,N):
    model.eval()
    total_correct = 0
    total_samples = 0
    for data in graphs:
            
        with torch.no_grad():
            i=0
        
            data = data.to(torch.device('cpu'))
            # print("prediction: ",i)
            out = model(data)
            i+=1
            pred = out.argmax(dim=1)
        
            num=int(data.x[0,14])
            suffixe=int(data.x[0,15])
            if suffixe!=-1:
                graphe_name='Graphe('+str(num)+')_'+str(suffixe)+'_features.json'
            else:
                graphe_name='Graphe('+str(num)+')_features.json'
            path='graphes_JSON_Complet/'+str(N)+'/test'
            print(graphe_name)
            print(path)


            add_prediction_to_json(pred,graphe_name,path)
            # print("actual: ",data.y)
            # print("pred: ",pred)
            total_correct += (pred == data.y).sum().item()
            total_samples += data.y.size(0)

    accuracy = total_correct / total_samples
    return accuracy

In [190]:
num_features=4
epochs=250
N=100

In [ ]:
for i in range(2,N+1):
    # loading graphs
    train_graphs, test_graphs = load_graphs_by_ncpu(i)
    train_loader = DataLoader(train_graphs, batch_size=4, shuffle=True)
    
    model = GraphSAGE(num_features,i).to(torch.device('cpu'))  
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # training 
    print("********Training on clusters with ",i," machines**********")
    train(model,epochs,train_loader,optimizer)
    # testing
    evaluate(model,test_graphs,i)
    #saving the model
    model_save_path = f'models/graphsage_clusters_{i}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Model for {i} clusters saved to {model_save_path}")

Loaded 44 training and 12 testing graphs for n_cpus=2
********Training on clusters with  2  machines**********
Epoch   1 | Loss: 0.9002 | Acc: 0.5823
Epoch  11 | Loss: 0.6404 | Acc: 0.5945
Epoch  21 | Loss: 0.6519 | Acc: 0.5875
Epoch  31 | Loss: 0.5732 | Acc: 0.6031
Epoch  41 | Loss: 0.5787 | Acc: 0.5962
Epoch  51 | Loss: 0.5413 | Acc: 0.6274
Epoch  61 | Loss: 0.5484 | Acc: 0.6205
Epoch  71 | Loss: 0.5003 | Acc: 0.6326
Epoch  81 | Loss: 0.4860 | Acc: 0.6655
Epoch  91 | Loss: 0.5023 | Acc: 0.6759
Epoch 100 | Loss: 0.4948 | Acc: 0.6915
Epoch 101 | Loss: 0.4735 | Acc: 0.6776
Epoch 111 | Loss: 0.4907 | Acc: 0.6984
Epoch 121 | Loss: 0.4605 | Acc: 0.7019
Epoch 131 | Loss: 0.4015 | Acc: 0.7314
Epoch 141 | Loss: 0.4184 | Acc: 0.7158
Epoch 151 | Loss: 0.4030 | Acc: 0.7764
Epoch 161 | Loss: 0.3345 | Acc: 0.7660
Epoch 171 | Loss: 0.3875 | Acc: 0.7504
Epoch 181 | Loss: 0.3335 | Acc: 0.7712
Epoch 191 | Loss: 0.3720 | Acc: 0.7556
Epoch 201 | Loss: 0.3499 | Acc: 0.7764
Epoch 211 | Loss: 0.3138 | Acc: